<a href="https://colab.research.google.com/github/easycloudapi/learn_gcp/blob/main/learning_resources/02_GCP_Compute_Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP Compute Instance Reference

1. Compute Engine API: https://cloud.google.com/compute/docs/reference/rest/v1?apix=true

# Spot VMs Vs Preemptible VMs Vs Standard VMs

ID   | Spot VMs | Preemptible VMs | Standard VMs
:--- | :--- | :--- | :---
1   | Spot VMs cost 60-91% less than standard VMs | Preemptible VMs similar like Spot VMs | standard VMs are costlier than Spot VMs
2   | Google can reclaim the resources by preempting Spot VMs at any time | Preemptible VMs have a maximum duration of 24 hours and do not offer availability guarantees. | Standrad VMs can't be preempted by Google Compute Engine
3   | Spot VMs are recommended only for fault-tolerant applications | Standard VMs have no such problems |
4   | Spot VMs will replace Preemptible VMs | Preemptible VMs have been deprecated | Standard VMs functionality will remain same.

# SSDs & Balanced Disks (Local Vs Persistent Disks)

![image](https://github.com/easycloudapi/learn_gcp/assets/108976294/85f0912f-65a3-4bfb-a790-a6751a57e9d2)

# Public Images

**Google provides public images that are maintained by open source communities and third-party vendors.** All Google Cloud projects have access to these images by default and can use them to create instances.

**Some OS images** are customized specifically to run on Compute Engine

```shell
gcloud compute images list
```

# Machine images

A machine image is a Compute Engine resource that stores all the configuration, metadata, permissions, and data from one or more disks required to create a virtual machine (VM) instance.

You can use a machine image in many system maintenance scenarios, such as instance creation, backup and recovery, and instance cloning.

**Custom images** capture the contents of a single disk, for example, a boot disk, which can be used to create new instances that are preconfigured with the apps that you need, so that you don’t have to configure public images from scratch.


**Machine images** are a more comprehensive resource that can contain multiple disks, as well as all of the information required to capture and create a new instance, including:

1. Instance properties (machine type, labels, volume mapping, network tags)
2. Data of all attached disks (one or multiple)
3. Instance metadata
4. Permissions, including the service account used to create the instance

# Instance templates

**An instance template lets you describe a VM instance.** You can then create groups of identical instances based on the template.

```shell
gcloud compute instance-templates list
```

![image](https://github.com/easycloudapi/learn_gcp/assets/108976294/f1d4bbf8-cd44-4d57-a38f-dc25864fc5ec)

# Multi-tenant Vs Sole-tenant

**Sole-tenancy** lets you have exclusive access to a sole-tenant node, which is a physical Compute Engine server that is dedicated to hosting only your project's VMs.

Use **sole-tenant nodes** to keep your VMs physically separated from VMs in other projects, or to group your VMs together on the same host hardware



![image](https://github.com/easycloudapi/learn_gcp/assets/108976294/71454c96-d4a3-4e5b-bd44-b710f4626432)

# (Managed) Instance Groups - MIGs

Instance groups let you organize VM instances or use them in a load-balancing backend service.

You can group existing instances or create a group based on an **instance template.**

# Snapshots

1. Snapshots are backups of persistent disks.
    1. **Snapshots:** Standard backup and disaster recovery; stored in a separate location from your disk
    2. **Instant snapshot:** Rapid restoration; stored in the same location as your disk
    3. **Archive snapshot:** Long-term storage for infrequently-accessed data; stored in a separate location from your disk
2. They’re commonly used to recover, transfer, or make data accessible to other resources in your project
3. We can schedule the snapshot

# Create New Compute Instance

1. How to **create Google VM Instance**: https://cloud.google.com/compute/docs/instances/create-start-instance
2. **Disk Info:** https://cloud.google.com/compute/docs/disks#pdspecs


## Using Python -

Pre-Requisite for Python:

1. Setup Working Folder
2. Setup Python Virtual ENV and Install Required Packages
2. Enable the GCP Compute API for the selected GCP Project

```shell
# cd $HOME
cd ~
mkdir demo && cd demo

python -m venv .venv
source .venv/bin/activate

# pip install --upgrade google-cloud-compute
```

### Client Libraries

1. The **Cloud Client Libraries** are the recommended way to access Google Cloud APIs programmatically.
2. The Cloud Client Libraries support accessing Google Cloud services in a way that significantly reduces the boilerplate code you have to write.
3. **The libraries provide high-level API abstractions so they're easier to understand**.


Python Client Library: https://cloud.google.com/python/docs/reference/

**Install Package for Client Libraries:** `pip install --upgrade google-cloud-compute`

**Use python client library:**  `from google.cloud import compute_v1`

[Click here for the link](https://github.com/easycloudapi/learn_gcp/blob/main/python_helpers/python_client_create_gcp_vm.py)

### Discovery API

machine-readable specification for describing and consuming REST APIs

1. https://www.googleapis.com/discovery/v1/apis/compute/v1/rest
2. https://github.com/googleapis/google-api-python-client

**Install Package for Discovery Api:** `pip install --upgrade google-api-python-client`

**Use this discovery build code:** `compute = googleapiclient.discovery.build("compute", "v1")`

[Click here for the link](https://github.com/easycloudapi/learn_gcp/blob/main/python_helpers/python_discovery_create_gcp_vm.py)

### Rest API


1. Core GCP Compute Rest API Documentation: https://cloud.google.com/compute/docs/reference/rest/v1/instances/insert
2. https://cloud.google.com/compute/docs/instances/create-start-instance

**Use base Url:**  `f'https://compute.googleapis.com/compute/v1/projects/{project}/zones/{zone}/instances'`

[Click here for the link](https://github.com/easycloudapi/learn_gcp/blob/main/python_helpers/python_rest_create_gcp_vm.py)

## Using Cloud Shell

```shell
# # Specify the VM Configs
VM_NAME1=demo-gcloud-vm-001  \
VM_NAME2=demo-gcloud-vm-002  \
ZONE1=us-central1-c  \
ZONE2=us-central1-f  \
IMAGE=debian-10-buster-v20200413  \
IMAGE_PROJECT=debian-cloud  \
MACHINE_TYPE=n2-standard-2  \
SIZE_GB=10  \
DISK_TYPE=pd-ssd  \
DISK_NAME=Demo-Disk
```

```shell
# # Create Cloud VM
gcloud compute instances create $VM_NAME1 \
--zone=$ZONE1 \
--image=$IMAGE  \
--image-project=$IMAGE_PROJECT \
--machine-type=$MACHINE_TYPE

# # Create Cloud VM with Additional Non-Boot Disk
gcloud compute instances create $VM_NAME2 \
--zone=$ZONE2 \
--image=$IMAGE  \
--image-project=$IMAGE_PROJECT \
--create-disk image=$IMAGE,image-project=$IMAGE_PROJECT,size=$SIZE_GB,type=$DISK_TYPE \
--create-disk=device-name=$DISK_NAME,size=$SIZE_GB,boot=false


# # Open the SSH of Cloud VM (Copy Below Code from GCP VM UI)
gcloud compute ssh --zone "us-west3-b" "<VM Name>" --tunnel-through-iap --project "<Project ID>"
# exit

# # Remove VM Machines
gcloud compute instances delete $VM_NAME1 --zone=$ZONE1
gcloud compute instances delete $VM_NAME2 --zone=$ZONE2
```

## Using Google Cloud UI

https://console.cloud.google.com/welcome?tutorial=compute_short_quickstart&_ga=2.249196100.-238573144.1707224980&_gac=1.121609338.1707225667.Cj0KCQiAzoeuBhDqARIsAMdH14GEM52x7poPip0EUmN49Ivf7ZheWM-AXDeHi3wXYQqX9lx6AxQpUHgaAiEeEALw_wcB&project=test-flask-api-372913

## Known Error

### Unavailable of Resource

![image](https://github.com/easycloudapi/learn_gcp/assets/108976294/372df431-1ba6-47b0-a4aa-e447cc6c478e)

# Other GCloud Useful Commands:

```shell
# gcloud compute zones list | grep us-central
gcloud compute zones list | FINDSTR us-central
gcloud compute images list | FINDSTR debian
gcloud compute disks list
gcloud compute disk-types list
gcloud compute machine-types list | FINDSTR  debian-11

gcloud compute instance-templates list
```